In [ ]:
#Load libraries\n",
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Set working directory
os.chdir("E:\EDW\Projects\Santander Prediction")
#os.getcwd()
#'E:\\EDW\\Projects\\Santander Prediction'

In [ ]:
%%time

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [ ]:
train_df.shape  #(200000, 202)
test_df.shape   #(200000, 201)
train_df.head()

In [ ]:
#Exploratory data analysis:
desc_train = train_df.describe()
desc_test = test_df.describe()

In [ ]:
##Missing Value Analysis

def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))


In [ ]:

#Missing values in train dataset
na_train = missing_data(train_df)
na_train.max(axis = 1)

In [ ]:

#Missing values in test dataset
na_test = missing_data(test_df)
na_test.max(axis = 1)

In [ ]:
#Scatter Plots

def plot_feature_scatter(df1, df2, features):
    i = 0
    sns.set_style('whitegrid')
    plt.figure()
    fig, ax = plt.subplots(5,5,figsize=(20,20))

    for feature in features:
        i += 1
        plt.subplot(5,5,i)
        plt.scatter(df1[feature], df2[feature], marker='+')
        plt.xlabel(feature, fontsize=15)
    plt.show();

In [ ]:
#Correlation Plot
features = train_df.columns
features = features.tolist()
features = features[2:]
feature_1 = features[0:25] #Splitting the features into smaller sets to plot correlation plot
feature_2 = features[26:51]
feature_3 = features[52:77]
feature_4 = features[78:103]
plot_feature_scatter(train_df[::20],test_df[::20], feature_1)

In [ ]:
plot_feature_scatter(train_df[::20],test_df[::20], feature_2)

In [ ]:
plot_feature_scatter(train_df[::20],test_df[::20], feature_3)

In [ ]:
# Compute the Correlation matrix
corr=train_df.iloc[:,:50].corr()
plt.figure(figsize=(20,20))
sns.heatmap(corr,cmap='Set1',annot=True)

In [ ]:
#Distribution of target variable

sns.countplot(train_df['target'], palette='tab10')

In [ ]:
#Density Plot:

def plot_predictor_distribution(df1, features):
    i = 0
    sns.set_style('whitegrid')
    plt.figure()
    fig, ax = plt.subplots(10,10,figsize=(20,20))

    for feature in features:
        i += 1
        plt.subplot(10,10,i)
        sns.distplot(df1[feature], hist=False)
        plt.xlabel(feature, fontsize=9)
        locs, labels = plt.xticks()
        plt.tick_params(axis='x', which='major', labelsize=6, pad=-6)
        plt.tick_params(axis='y', which='major', labelsize=6)
    plt.show();

In [ ]:
features = train_df.columns.values[102:202]
plot_predictor_distribution(train_df, features)

In [ ]:
#Density Plot:

def plot_feature_distribution(df1, df2, label1, label2, features):
    i = 0
    sns.set_style('whitegrid')
    plt.figure()
    fig, ax = plt.subplots(10,10,figsize=(20,20))

    for feature in features:
        i += 1
        plt.subplot(10,10,i)
        sns.distplot(df1[feature], hist=False,label=label1)
        sns.distplot(df2[feature], hist=False,label=label2, color='r')
        plt.xlabel(feature, fontsize=9)
        locs, labels = plt.xticks()
        plt.tick_params(axis='x', which='major', labelsize=6, pad=-6)
        plt.tick_params(axis='y', which='major', labelsize=6)
    plt.show();

In [ ]:
t0 = train_df.loc[train_df['target'] == 0]
t1 = train_df.loc[train_df['target'] == 1]
features = train_df.columns.values[2:102]
plot_feature_distribution(t0, t1, '0', '1', features)

In [ ]:
features = train_df.columns.values[103:202]
plot_feature_distribution(t0, t1, '0', '1', features)

In [ ]:

features = train_df.columns.values[2:102]
plot_feature_distribution(train_df, test_df, 'train', 'test', features)

In [ ]:
%%time
#Feature selection1::

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

X = train_df.iloc[:,2:202]  #independent columns
y = train_df.iloc[:,1]    #target column

#apply SelectKBest class to extract top 20 best features
bestfeatures = SelectKBest(score_func=f_classif, k=50)
fit_1 = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit_1.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
feat_importance_1 = pd.concat([dfcolumns,dfscores],axis=1)
feat_importance_1.columns = ['Variable','Score']  #naming the dataframe columns
feat_importance_1 = feat_importance_1.sort_values(by = ['Score'], ascending = False)
print(feat_importance_1.nlargest(50,'Score'))

Var_to_model_1 = feat_importance_1.Variable[0:50] #choosing first 50 variables with high importance values

In [ ]:
#feature Selection2:

from sklearn.ensemble import ExtraTreesClassifier

X = train_df.iloc[:,2:202]  #independent columns

y = train_df.iloc[:,1]    #target column
y=y.astype('int64')
feat_model = ExtraTreesClassifier( )
feat_model.fit(X,y)
print(feat_model.feature_importances_)
feat_importance_2 = pd.DataFrame(feat_model.feature_importances_, index=X.columns)
feat_importance_2['Variable'] = feat_importance_2.index
feat_importance_2.columns = ['Score', 'Variable']
feat_importance_2 = feat_importance_2.sort_values(by = ['Score'], ascending = False)
feat_importance_2
Var_to_model_2 = feat_importance_2.Variable[0:50]
#dfscores = pd.DataFrame(fit_1.scores_)
#dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
#feat_importance_1 = pd.concat([dfcolumns,dfscores],axis=1)
#feat_importance_1.columns = ['Variable','Score']  #naming the dataframe columns
#print(feat_importance_1.nlargest(50,'Score'))

In [ ]:
#Variable selection from 2 methods:
Var_to_model_1 = Var_to_model_1.tolist( )
Var_to_model_2 = Var_to_model_2.tolist( )
Var_to_model=[i for i in Var_to_model_1 if i in Var_to_model_2]
Var_to_model #common variables in both variable selection methods

In [ ]:
len(Var_to_model)
print(Var_to_model)

In [ ]:
#Field selection from teh original data set

train_df_1 = train_df[['ID_code', 'target']]
train_df_2 = train_df[Var_to_model]
train_df_1 = pd.concat([train_df_1, train_df_2], axis = 1)

In [ ]:
test_df_1 = test_df[['ID_code']]
test_df_2 = test_df[Var_to_model]
test_df_1 = pd.concat([test_df_1, test_df_2], axis = 1)

In [ ]:
#Outlier Analysis:

def plot_outlier(df1, features):
    i = 0
    sns.set_style('whitegrid')
    plt.figure()
    fig, ax = plt.subplots(9,5,figsize=(20,20))

    for var in features:
        i += 1
        plt.subplot(9,5,i)
        sns.boxplot(x=df1[var])
        plt.xlabel(var, fontsize=9)
        locs, labels = plt.xticks()
        plt.tick_params(axis='x', which='major', labelsize=6, pad=-6)
        plt.tick_params(axis='y', which='major', labelsize=6)
    plt.show();

In [ ]:
plot_outlier(train_df_1, Var_to_model)

In [ ]:
#remove Outliers:


for i in Var_to_model:
    q75, q25 = np.percentile(train_df_1.loc[:, i], [75 ,25])
    iqr = q75 - q25
    min = q25 - (iqr*1.5)
    max = q75 + (iqr*1.5)
    print(i)
    print(min)
    print(max)

    train_df_1 = train_df_1.drop(train_df_1[train_df_1.loc[:,i] < min].index)
    train_df_1 = train_df_1.drop(train_df_1[train_df_1.loc[:,i] > max].index)

In [ ]:
#After removing outliers:
plot_outlier(train_df_1, Var_to_model)
train_df_1.shape

In [ ]:
#Stratified sampling

from sklearn.model_selection import StratifiedShuffleSplit

ss = StratifiedShuffleSplit(n_splits=10, test_size=0.33, random_state= 100)

for train_index, test_index in ss.split(train_df_1, train_df_1['target']):
    train = train_df_1.iloc[train_index]
    test = train_df_1.iloc[test_index]
    
#Target variable distribution after stratified sampling
sns.countplot(train['target'], palette='tab10')
sns.countplot(test['target'], palette='tab10')

In [ ]:

#Model _1
#Logistic regression:

def logistic_regression(y,X):
    global result
    logit_model=sm.Logit(y,X)
    result=logit_model.fit()
    print(result.summary2())
    
    

In [ ]:
#Model 2:
#Decision tree:

def decision_tree_mod(X_train,y_train,X_test):
    global y_pred, clf
    clf = DecisionTreeClassifier()

    # Train Decision Tree Classifer
    clf = clf.fit(X_train,y_train)

    #Predict the response for test dataset
    y_pred = clf.predict(X_test)
    
   

In [ ]:
#Model 3:
def random_forest_mod(X,y,x_test):
    global y_test, clf
    # Create a random forest Classifier. By convention, clf means 'Classifier'
    clf = RandomForestClassifier(n_estimators=100, n_jobs=2, random_state=0)

    # Train the Classifier to take the training features and learn how they relate
    # to the training y (the species)
    clf.fit(X, y)
    
    # Apply the Classifier we trained to the test data (which, remember, it has never seen before)
    y_test = clf.predict(x_test)
    

In [1]:
#Build confusion matrix

def confusion_matrix(yy,xx):
    CM = pd.crosstab(yy,xx)
    global TN, FN, TP, FP

    #let us save TP, TN, FP, FN
    TN = CM.iloc[0,0]
    FN = CM.iloc[1,0]
    TP = CM.iloc[1,1]
    FP = CM.iloc[0,1]

    print("True Negative", TN)
    print("True Positive", TP)
    print("False Negative", FN)
    print("False Positive", FP)

    #check accuracy of model
    #accuracy_score(y_test, y_pred)*100
    print("Accuracy : ", ((TP+TN)*100)/(TP+TN+FP+FN))
    
def Metrics(TP,TN,FP,FN):
    ACC = ((TP+TN)*100)/(TP+TN+FP+FN)
    TPR = TP/(TP+FN)
    FPR = FP/(FP+TN)
    TNR = TN/(TN+FP)
    FNR = FN/(FN+TP)
    Precision = TP/(TP+FP)
    F1 = 2*((Precision*TPR)/(Precision+TPR))
    
    print("Accuracy :", ACC)
    print("Sensitivity or Recall/TPR :", TPR)
    print("Specificity or TNR :", TNR)
    print("Fall-Out or FPR :", FPR)
    print("Miss rate or False Negative rate :", FNR)
    print("Precision :" , Precision)
    print("F1 Score: " , F1)




In [ ]:
#Logistic regression with stratified samples
logistic_regression(train['target'],train[Var_to_model])
#Predict test data
   
test['predicted_prob'] = result.predict(test[Var_to_model])
test['PredictedVal'] = 1
test.loc[test.predicted_prob < 0.5, 'PredictedVal'] = 0
    

In [ ]:
confusion_matrix(test['target'], test['PredictedVal'])
Metrics(TP,TN,FP,FN)

In [ ]:
#ROC Curve
X_test = test[Var_to_model]

log_ROC_auc = roc_auc_score(test['target'], result.predict(X_test))
fpr, tpr, threshold = roc_curve(test['target'], test['predicted_prob'])
plt.figure()
plt.plot(fpr,tpr, label = "ROC Curve ( area = %0.2f )" %log_ROC_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic ')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#Decision tree with Stratified samples
decision_tree_mod(train[Var_to_model], train['target'], test[Var_to_model])
confusion_matrix(test['target'], y_pred)
Metrics(TP,TN,FP,FN)

In [ ]:
#ROC Curve
X_test = test[Var_to_model]

log_ROC_auc = roc_auc_score(test['target'], clf.predict(X_test))
fpr, tpr, threshold = roc_curve(test['target'], y_pred)
plt.figure()
plt.plot(fpr,tpr, label = "ROC Curve ( area = %0.2f )" %log_ROC_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic ')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#Random Forest with Stratified samples
random_forest_mod(train[Var_to_model], train['target'], test[Var_to_model])
confusion_matrix(test['target'], y_test)
Metrics(TP,TN,FP,FN)

In [ ]:
#ROC Curve
X_test = test[Var_to_model]

log_ROC_auc = roc_auc_score(test['target'], clf.predict(X_test))
fpr, tpr, threshold = roc_curve(test['target'], y_test)
plt.figure()
plt.plot(fpr,tpr, label = "ROC Curve ( area = %0.2f )" %log_ROC_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic ')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#Undersampling
# Class count
count_class_0, count_class_1 = train_df_1.target.value_counts()

# Divide by class
df_class_0 = train_df_1[train_df_1['target'] == 0]
df_class_1 = train_df_1[train_df_1['target'] == 1]

df_class_0_under = df_class_0.sample(count_class_1, replace = False)
df_under = pd.concat([df_class_0_under, df_class_1], axis=0)

In [ ]:
for train_index, test_index in ss.split(df_under, df_under['target']):
    train = df_under.iloc[train_index]
    test = df_under.iloc[test_index]

In [ ]:
#OverSampling
# Class count
count_class_0, count_class_1 = train_df_1.target.value_counts()

# Divide by class
df_class_0 = train_df_1[train_df_1['target'] == 0]
df_class_1 = train_df_1[train_df_1['target'] == 1]

df_class_1_over = df_class_1.sample(count_class_0//2, replace = True)
df_over = pd.concat([df_class_1_over, df_class_0], axis=0)

In [ ]:
for train_index, test_index in ss.split(df_under, df_under['target']):
    train = df_under.iloc[train_index]
    test = df_under.iloc[test_index]

In [ ]:
#Logistic regression with stratified over-sampled data
logistic_regression(train['target'],train[Var_to_model])
#Predict test data
   
test['predicted_prob'] = result.predict(test[Var_to_model])
test['PredictedVal'] = 1
test.loc[test.predicted_prob < 0.5, 'PredictedVal'] = 0
confusion_matrix(test['target'], test['PredictedVal'])
Metrics(TP,TN,FP,FN)

In [ ]:
#Decision tree with stratified over-sampled data
decision_tree_mod(train[Var_to_model], train['target'], test[Var_to_model])
confusion_matrix(test['target'], y_pred)
Metrics(TP,TN,FP,FN)

In [ ]:
#Random Forest with stratified over-sampled data
random_forest_mod(train[Var_to_model], train['target'], test[Var_to_model])
confusion_matrix(test['target'], y_test)
Metrics(TP,TN,FP,FN)